<a href="https://colab.research.google.com/github/sochachai/Transformer_Analysis/blob/main/5_digits_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load packages

In [2]:
# check directory
!pwd

/content


In [3]:
# confirm the assistant py file has been uploaded to the correct directory
!ls

my_transformer_utils.py  sample_data


In [4]:
# check Python version
!python --version

Python 3.10.12


In [5]:
# install torch
!pip install torch==2.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [7]:
# check torch version
import torch
print(torch.__version__)

2.2.0+cu121


In [8]:
import copy
import torch
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
# Issue caused by torch version described below:
# the original package should be pyitcast.transformer_utils
# the problem is that pyitcast.transformer_utils relies on an old version of torch (1.3.1 should work)
# and causes the error "IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python..."
# but the installation of an old version of torch that match pyitcast.transformer is not trivial
# versions of 1.11.0 or after is not compatible with pyitcast.transformer and installation of them will not
# solve the issue
# To solve this issue:
# download the pyitcast.transformer_utils (open by clicking the error message) as a py file
# modify the last line of the SimpleLossCompute class from "return loss.data[0] * norm"
# to "return loss.data * norm"

from my_transformer_utils import Batch
from my_transformer_utils import run_epoch
from my_transformer_utils import greedy_decode
from my_transformer_utils import get_std_opt # get_std_opt is based on Adam optimizer
from my_transformer_utils import LabelSmoothing # offset human label errors to prevent overfitting
from my_transformer_utils import SimpleLossCompute # calculate loss after smoothing, use cross_entropy_loss


Construct the class of Embeddings and Positional Encoding.

In [9]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        '''
        :param d_model: embedding dimension
        :param vocab: size of vocabulary
        '''
        # Initialization
        super(Embeddings, self).__init__()
        # Defrine a word embedding object
        self.lut = nn.Embedding(vocab, d_model)
        # Instantiate d_model
        self.d_model = d_model

    def forward(self, x):
        '''
        :param x: tensor representing the original text
        '''
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len = 5000):
        '''
        :param d_model: dimension of the encoding
        :param dropout: dropout rate from 0 to 1
        :param max_len: the maximum length of a sentence
        '''
        # Inherit the initialization of nn.Module
        super(PositionalEncoding, self).__init__()

        # Objectify dropout
        self.dropout = nn.Dropout(p=dropout)

        # Inherit a positional encoder matrix, max_len * d_model
        pe = torch.zeros(max_len, d_model)

        # Inherit an absolute position matrix, max_len * 1
        position = torch.arange(0, max_len).unsqueeze(1)

        # Define the conversion matrix, initialization with gap = 2
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0)/d_model))

        # Copy the absolute position matrix to the positional encoder matrix
        # by sine and cosine wave according to the parity of column indices
        pe[:, 0::2] = torch.sin(position * div_term) # even indiced columns are imputed by sine
        pe[:, 1::2] = torch.cos(position * div_term) # odd indiced columns are imputed by cosine

        # Extend pe to 3-dimensional tensor
        pe = pe.unsqueeze(0)

        # Register pe to a buffer, the buffer is not a parameter of the class
        # the buffer will not be updated along with the model update
        # but it can be loaded along with the model
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        :param x: Tensor of text
        :return: x + the positional encoding
        '''
        # Shrink the size of pe to save storage
        # by converting the second dimension, i.e. the dimension of max_len
        # to the size of the sentence len of x, i.e. the second dimension of x
        x = x + Variable(self.pe[:,:x.size(1)], requires_grad = False) # False: pe will not be updated
        return self.dropout(x)

def attention(query, key, value, mask = None, dropout = None):
    '''
    :param query: vectorized original text
    :param key: key words of text
    :param value: the original value of key, summarization of query
    :param mask: hide words to avoid data leakage
    :param dropout: dropout rate of neural network
    :return:
    '''
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9) # compare each position with 0

    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)

    return torch.matmul(p_attn, value), p_attn

def clones(module, N):
    '''
    :param module: one attention layer
    :param N: the number of module
    '''
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)]) # deepcopy uses a different memory

class MultiHeadedAttention(nn.Module):
    def __init__(self, head, embedding_dim, dropout = 0.1):
        '''
        :param head: the number of heads
        :param embedding_dim: the embedding dimension
        :param dropout: default dropout rate set to 0.1
        '''
        # Inherit the initialization
        super(MultiHeadedAttention, self).__init__()
        # head must be an integral factor of embedding_dim
        assert embedding_dim % head == 0
        # each head is assigned with the following dimension
        self.d_k = embedding_dim // head # division in the integral domain Z
        # substantiate head
        self.head = head
        # create linear layers, we need 4 of them for Q, K, V and the final connection
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)
        # create the attention and dropout rate
        self.attn = None
        self.dropout = nn.Dropout(p = dropout)

    def forward(self, query, key, value, mask = None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        batch_size = query.size(0)

        # We have 4 linears and only zipping the first 3 with Q, K, V, the last linear is not used.
        # view(batch_size, -1, self.head, self.d_k).transpose(1,2)
        # is not the same with view(batch_size, self.head, -1, self.d_k)
        # self.head and self.d_k should be neighboring to get embedding_dim in order for the tensor
        # to interpret the relationship between the meaning of words of their positions in a sentence

        query, key, value = \
            [model(x).view(batch_size, -1, self.head, self.d_k).transpose(1,2)
             for model, x in zip(self.linears, (query, key, value))]

        x, self.attn = attention(query, key, value, mask = mask, dropout = self.dropout)

        # Reshape x
        # must use contiguous method after the transpose before the view method
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.head * self.d_k)

        # Pass x to the 4th linear layer
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        '''
        :param d_model: embedding dimension
        :param d_ff: transitional dimension
        :param dropout: default dropout set to 0.1
        '''
        super(PositionwiseFeedForward, self).__init__()
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w2(self.dropout(F.relu(self.w1(x))))

class LayerNorm(nn.Module):
    def __init__(self, features, eps = 1e-6):
        '''
        :param features: embedding dimension
        :param eps: avoid zero denominator
        '''
        super(LayerNorm, self).__init__()
        # nn.Parameter formats the variables to intrinsic parameters
        # they will be updated along with the model in contrast with buffer
        self.a2 = nn.Parameter(torch.ones(features))
        self.b2 = nn.Parameter(torch.zeros(features))
        # initialization of eps
        self.eps = eps

    def forward(self, x):
        '''
        :param x: the output of previous layer
        :return: numerical standardized x
        '''
        mean = x.mean(-1, keepdim = True)
        std = x.std(-1, keepdim = True)
        return self.a2 * (x - mean) / (std + self.eps) + self.b2

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout = 0.1):
        '''
        :param size: embedding size
        :param dropout: deactivation of neurons to avoid overfitting
        '''
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, sublayer):
        '''
        :param x: the output of previous layer
        :param sublayer: a function, e.g. Multihead_Attention, PositionwiseFeedForward etc.
        :return: x plus sublayer functioning on normed x with dropout
        '''
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        '''
        :param size: embedding dimension
        :param self_attn: attention
        :param feed_forward: positionwise feed forward
        :param dropout: avoid overfitting
        '''
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        '''
        :param x: output of previous layer
        :param mask: tensor mask to prevent data leakage
        '''
        # input matrix followed by operating function, returns an output matrix
        # sublayer(x, function) will return function(x)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) # the forward function of multihead attention
        return self.sublayer[1](x, self.feed_forward) # the forward function of pointwise feedforward

class Encoder(nn.Module):
    # Encoder is a collection of Encoder Layers
    def __init__(self, layer, N):
        '''
        :param layer: one encoder layer
        :param N: the number of encoder layers
        '''
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        for layer in self.layers: x = layer(x, mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        '''
        :param size: embedding dimension
        :param self_attn: masked multihead attention
        :param src_attn: multihead attention
        :param feed_forward: pointwise feed forward
        :param dropout: avoid overfitting
        '''
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        # 3 sublayer for a decoder layer
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, source_mask, target_mask):
        '''
        :param x: output of previous layer
        :param memory: result of encoder
        :param source_mask: delete unnecessary info to improve model performance
        :param target_mask: hide info to prevent data leakage
        :return: output tensor
        '''
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, target_mask))
        # for second layer, Q = x, K = V = m
        x = self.sublayer[1](x, lambda x: self.self_attn(x, m, m, source_mask))
        return self.sublayer[2](x, self.feed_forward)

class Decoder(nn.Module):
    # Decoder is a collection of Decoder Layers
    def __init__(self, layer, N):
        '''
        :param layer: decoder layer
        :param N: the number of decoder layers
        '''
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, source_mask, target_mask):
        for layer in self.layers:
            x = layer(x, memory, source_mask, target_mask)
        return self.norm(x)

class Generator(nn.Module):
    def __init__(self, d_model, vocab_size):
        '''
        :param d_model: embedding dimension
        :param vocab_size: the size of the vocabulary
        '''
        super(Generator, self).__init__()
        self.project = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return F.log_softmax(self.project(x), dim = -1)

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, source_embed, target_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = source_embed
        self.tgt_embed = target_embed
        self.generator = generator

    def forward(self, source, target, source_mask, target_mask):
        # encoded source used as memory in decode function
        return self.decode(self.encode(source, source_mask), source_mask,
                           target, target_mask)

    def encode(self, source, source_mask):
        return self.encoder(self.src_embed(source), source_mask)

    def decode(self, memory, source_mask, target, target_mask):
        # embedded target as x in the decoder function
        return self.decoder(self.tgt_embed(target), memory, source_mask, target_mask)

def make_model(source_vocab, target_vocab, N=6,\
               d_model=512, d_ff=2048, head=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(head, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn),c(ff), dropout), N),
        # note the order of vocab_size and d_model;
        # nn.Embedding is not the same with Embeddings;
        # check Embedding_Encoder.py for more;
        nn.Sequential(Embeddings(d_model, source_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, target_vocab), c(position)),
        Generator(d_model, target_vocab)
    )
    for p in model.parameters():
        if p.dim() > 1: nn.init.xavier_uniform_(p) # initialization: make p uniformly sampled; check xavier_uniform for details
    return model



'''
Task: Test the language model's capability to copy a sequence of numbers
      Input a list of numbers, the output should be the identical list of numbers
SubTask: Write a data generator to generate sample data for model testing
'''

# use a function to generate data
def data_generator(V, batch_size, num_batch):
    '''
    :param V: the maximal data value + 1
    :param batch_size: sample data size of one round of training after which model parameters are updated
    :param num_batch: number of rounds of training
    '''
    for i in range(num_batch):
        # data value from 1 to V-1, with data matrix shape = batch_size times 10
        data = torch.from_numpy(np.random.randint(1, V, size = (batch_size, 10)))

        # set starting position label
        data[:, 0] = 1

        # for a copy task source data and target data should be the same
        # no gradient calculation involved
        source = Variable(data, requires_grad = False)
        target = Variable(data, requires_grad = False)
        yield Batch(source, target)


def run(model, loss, epochs=10):
    '''
    :param model: model
    :param loss: loss function
    :param epochs: number of rounds of training
    '''
    for epoch in range(epochs):
        # train model, update model parameters
        model.train()
        run_epoch(data_generator(V, 200, 8), model, loss)
        # evaluate model, no parameters update
        model.eval()
        run_epoch(data_generator(V, 50, 8), model, loss)


Instantiate a model.

In [28]:
# Instantiate variables
V = 6
batch_size = 20
num_batch = 30

# get model
model = make_model(V, V, N = 6)

# get optimizer
model_optimizer = get_std_opt(model)

# get smooth criterion
criterion = LabelSmoothing(size = V, padding_idx = 0, smoothing = 0.0)

# get loss function
# model as an EncoderDecoder whose last element is a generator object
loss = SimpleLossCompute(model.generator, criterion, model_optimizer)


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


10 epochs of model training.

In [29]:
# model training
epochs = 10
if __name__ == '__main__':
    run(model, loss, epochs)

Epoch Step: 1 Loss: 2.037391 Tokens per Sec: 157.054108
Epoch Step: 1 Loss: 1.850224 Tokens per Sec: 152.114258
Epoch Step: 1 Loss: 1.716174 Tokens per Sec: 171.577301
Epoch Step: 1 Loss: 1.431418 Tokens per Sec: 179.027100
Epoch Step: 1 Loss: 1.482326 Tokens per Sec: 170.137909
Epoch Step: 1 Loss: 1.316452 Tokens per Sec: 179.484711
Epoch Step: 1 Loss: 1.372549 Tokens per Sec: 173.278351
Epoch Step: 1 Loss: 1.254976 Tokens per Sec: 168.027817
Epoch Step: 1 Loss: 1.343951 Tokens per Sec: 178.864838
Epoch Step: 1 Loss: 1.264327 Tokens per Sec: 148.663406
Epoch Step: 1 Loss: 1.330258 Tokens per Sec: 169.095764
Epoch Step: 1 Loss: 1.178638 Tokens per Sec: 180.879425
Epoch Step: 1 Loss: 1.289549 Tokens per Sec: 175.482727
Epoch Step: 1 Loss: 1.082457 Tokens per Sec: 181.511505
Epoch Step: 1 Loss: 1.226902 Tokens per Sec: 177.647034
Epoch Step: 1 Loss: 1.094218 Tokens per Sec: 179.631683
Epoch Step: 1 Loss: 1.248441 Tokens per Sec: 183.396729
Epoch Step: 1 Loss: 0.983338 Tokens per Sec: 181

Test the model's capibility of copying the original sequence with greedy decode after 10 epochs of training.

In [30]:
def test_model(an_integer_sequence_of_size_5):
    # greedy decode
    # enter evaluation mode
    model.eval()

    # get source input
    source = Variable(torch.LongTensor([an_integer_sequence_of_size_5]))

    # get source mask
    # 1 for no masking
    source_mask = Variable(torch.ones(1, 1, 5))

    # get result
    result = greedy_decode(model, source, source_mask, max_len=5, start_symbol=1)
    print(f"The source numeric sequence is:\n {source}")
    print(f"The resulting numeric sequence after {epochs} epochs of training is:\n {result}")

test_model([1,  3,  2,  5,  4])

The source numeric sequence is:
 tensor([[1, 3, 2, 5, 4]])
The resulting numeric sequence after 10 epochs of training is:
 tensor([[1, 3, 2, 5, 3]])


Test the model's capibility of copying the original sequence with greedy decode after an extra of 20 epochs of training.

In [31]:
# model training
epochs = 20
if __name__ == '__main__':
    run(model, loss, epochs)


Epoch Step: 1 Loss: 1.239350 Tokens per Sec: 162.733932
Epoch Step: 1 Loss: 0.908190 Tokens per Sec: 171.653259
Epoch Step: 1 Loss: 0.996257 Tokens per Sec: 181.329926
Epoch Step: 1 Loss: 0.741328 Tokens per Sec: 148.147736
Epoch Step: 1 Loss: 0.963049 Tokens per Sec: 166.191589
Epoch Step: 1 Loss: 0.628003 Tokens per Sec: 167.297852
Epoch Step: 1 Loss: 0.933253 Tokens per Sec: 169.863144
Epoch Step: 1 Loss: 0.629011 Tokens per Sec: 151.229950
Epoch Step: 1 Loss: 0.831857 Tokens per Sec: 168.031021
Epoch Step: 1 Loss: 0.548762 Tokens per Sec: 174.622253
Epoch Step: 1 Loss: 0.777321 Tokens per Sec: 168.225006
Epoch Step: 1 Loss: 0.414798 Tokens per Sec: 180.782944
Epoch Step: 1 Loss: 0.658169 Tokens per Sec: 177.783142
Epoch Step: 1 Loss: 0.373356 Tokens per Sec: 158.143326
Epoch Step: 1 Loss: 0.620513 Tokens per Sec: 173.372635
Epoch Step: 1 Loss: 0.250569 Tokens per Sec: 157.542694
Epoch Step: 1 Loss: 0.555389 Tokens per Sec: 172.896469
Epoch Step: 1 Loss: 0.188450 Tokens per Sec: 173

In [32]:
test_model([1,  3,  2,  5,  4])

The source numeric sequence is:
 tensor([[1, 3, 2, 5, 4]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 3, 2, 5, 5]])


In [33]:
test_model([2,  5,  1,  4,  3])

The source numeric sequence is:
 tensor([[2, 5, 1, 4, 3]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 5, 1, 4, 3]])


In [34]:
test_model([2,  3,  1,  5, 4])

The source numeric sequence is:
 tensor([[2, 3, 1, 5, 4]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 3, 1, 5, 5]])


In [35]:
test_model([[1,2,3,4,5]])

The source numeric sequence is:
 tensor([[[1, 2, 3, 4, 5]]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 2, 4, 3, 5]])


In [36]:
test_model([[5,4,3,2,1]])

The source numeric sequence is:
 tensor([[[5, 4, 3, 2, 1]]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 2, 4, 3, 5]])


In [37]:
test_model([[1,3,5,2,4]])

The source numeric sequence is:
 tensor([[[1, 3, 5, 2, 4]]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 2, 4, 3, 5]])


In [38]:
test_model([[2,3,4,2,3]])

The source numeric sequence is:
 tensor([[[2, 3, 4, 2, 3]]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 2, 3, 2, 3]])


In [39]:
test_model([[1,1,1,1,1]])

The source numeric sequence is:
 tensor([[[1, 1, 1, 1, 1]]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 1, 1, 1, 1]])


In [40]:
test_model([[2,2,2,2,2]])

The source numeric sequence is:
 tensor([[[2, 2, 2, 2, 2]]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 2, 2, 2, 2]])


In [41]:
test_model([[0,1,2,3,4]])

The source numeric sequence is:
 tensor([[[0, 1, 2, 3, 4]]])
The resulting numeric sequence after 20 epochs of training is:
 tensor([[1, 2, 3, 4, 3]])
